# Exercise 3

## Imports

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import time
import os
import transformers
import optuna
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, balanced_accuracy_score
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Data loading

In [ ]:
ag_news_train_df = pd.read_csv('data/ag_news_train.csv')
ag_news_test_df = pd.read_csv('data/ag_news_test.csv')
amazon_reviews_df = pd.read_csv('data/cleaned_amazon_reviews.csv')

In [ ]:
ag_news_train_df.shape

In [ ]:
ag_news_test_df.shape

In [ ]:
ag_news_train_df.head()

In [ ]:
amazon_reviews_df.shape

In [ ]:
amazon_reviews_df.head()

In [ ]:
amazon_reviews_df.isna().sum()

In [ ]:
amazon_reviews_df.sentiments.value_counts().plot(kind='bar')

Unbalanced target

In [ ]:
ag_news_train_df['Class Index'].value_counts().plot(kind='bar')

Balanced target

#### Distribution of review lengths

In [ ]:
plt.hist(amazon_reviews_df.cleaned_review_length)

#### Distribution of the news article lengths

In [ ]:
plt.hist(ag_news_train_df.Description.str.len())

In [ ]:
ag_news_train_df.isna().any()

#### Removing NaNs

In [ ]:
amazon_reviews_df.dropna(inplace=True)

### Splitting amazon reviews into train and test set

In [ ]:
X_amazon_reviews = amazon_reviews_df.drop(columns=['sentiments'])
y_amazon_reviews = amazon_reviews_df['sentiments']

In [ ]:
X_train_reviews, X_test_reviews, y_train_reviews, y_test_reviews = train_test_split(X_amazon_reviews,
                                                                                   y_amazon_reviews,
                                                                                   random_state=42,
                                                                                   test_size=0.2)

In [ ]:
X_train_reviews, X_valid_reviews, y_train_reviews, y_valid_reviews = train_test_split(X_train_reviews,
                                                                                     y_train_reviews,
                                                                                     random_state=42,
                                                                                     test_size = 0.33)

### Splitting ag news into X and y's

Merging Title and Description into one field, "Text"

In [ ]:
ag_news_train_df['Text'] = ag_news_train_df[['Title', 'Description']].agg(' '.join,axis=1)
ag_news_test_df['Text'] = ag_news_test_df[['Title', 'Description']].agg(' '.join,axis=1)

In [ ]:
ag_news_train_df['Text']

In [ ]:
X_train_news = ag_news_train_df['Text']
y_train_news = ag_news_train_df['Class Index']

X_test_news = ag_news_test_df['Text']
y_test_news = ag_news_test_df['Class Index']

In [ ]:
X_train_news, X_valid_news, y_train_news, y_valid_news = train_test_split(X_train_news,
                                                                          y_train_news,
                                                                          random_state=42,
                                                                          test_size = 0.33)

## Naive feature extraction

### Uni-Bigram vectorizer

Because documents are quite short on average, we used the uni-bigram vectorizer (contains both unigrams and bigrams)

Following pieces of code were taken from Sklearn's text feature extraction tutorial. (https://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=r'\b\w+\b', min_df=1)

In [ ]:
X_train_reviews_tr = bigram_vectorizer.fit_transform(X_train_reviews.cleaned_review)
X_valid_reviews_tr = bigram_vectorizer.transform(X_valid_reviews.cleaned_review)
X_test_reviews_tr = bigram_vectorizer.transform(X_test_reviews.cleaned_review)

In [ ]:
bigram_vectorizer.get_feature_names_out()

In [ ]:
X_train_news_tr = bigram_vectorizer.fit_transform(X_train_news)
X_valid_news_tr = bigram_vectorizer.transform(X_valid_news)
X_test_news_tr = bigram_vectorizer.transform(X_test_news)

## Training shallow models

Balanced accuracy score is used to macro average across classes.

For hyperparam tuning only a half of training data will be used for performance reasons.

In [ ]:
np.random.seed(42)
reviews_train_idx = np.random.choice(y_train_reviews.shape[0], 
                                     replace=False, size=y_train_reviews.shape[0]//2)
news_train_idx = np.random.choice(y_train_news.shape[0], 
                                     replace=False, size=y_train_news.shape[0]//8)

In [ ]:
X_train_reviews_half = X_train_reviews_tr[reviews_train_idx,:]
y_train_reviews_half = y_train_reviews.iloc[reviews_train_idx]

X_train_news_16th = X_train_news_tr[news_train_idx,:]
y_train_news_16th = y_train_news.iloc[news_train_idx]

X_train_news_8th = X_train_news_tr[news_train_idx,:]
y_train_news_8th = y_train_news.iloc[news_train_idx]

#### Checking if models are saved

In [ ]:
best_model_nn_reviews_is_saved = os.path.exists('models/best_model_nn_reviews.sav')
best_model_nn_news_is_saved = os.path.exists('models/best_model_nn_news.sav')
best_model_svm_reviews_is_saved = os.path.exists('models/best_model_svm_reviews.sav')
best_model_svm_news_is_saved = os.path.exists('models/best_model_svm_news.sav')


### Shallow NN (One layer perceptron)

#### Reviews dataset

In [ ]:
available_activations = ['relu', 'logistic', 'tanh']
available_solvers = ['adam', 'lbfgs']
def objective(trial: optuna.trial.Trial):
    size_of_hidden_layer = trial.suggest_int('size_of_hidden_layer', 10, 100)
    alpha = trial.suggest_float('alpha', 0.0001, 0.001)
    activation = trial.suggest_categorical('activation', 
                                           choices=available_activations)
    solver = trial.suggest_categorical('solver', 
                                           choices=available_solvers)
    
    print(f"Trying: {size_of_hidden_layer}, {alpha}, {activation}, {solver}")
    model = MLPClassifier(hidden_layer_sizes=tuple([size_of_hidden_layer]),
                          alpha=alpha,
                          activation = activation,
                          max_iter=20,
                          solver=solver,
                          verbose=True,
                          learning_rate='adaptive',
                          random_state=42)
    model.fit(X_train_reviews_half, y_train_reviews_half)
    y_pred_probas = model.predict_proba(X_valid_reviews_tr)
    return log_loss(y_valid_reviews, y_pred_probas)


if not best_model_nn_reviews_is_saved:
    study_nn_reviews = optuna.create_study(direction='minimize')
    # optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study_nn_reviews.optimize(objective, n_trials=10,show_progress_bar=True,n_jobs=-1)

    study_nn_reviews.best_params

{'size_of_hidden_layer': 85,
 'alpha': 0.00045434026867030927,
 'activation': 'logistic',
 'solver': 'adam'}

In [ ]:
if not best_model_nn_reviews_is_saved:
    best_model_nn_reviews = MLPClassifier(hidden_layer_sizes=(85,), 
                              alpha=0.00045434026867030927,
                              solver='adam',
                              activation='logistic',
                                random_state=42,
                               learning_rate='adaptive',
                               verbose=True,
                              max_iter=50,
                          )
    best_model_nn_reviews.fit(X_train_reviews_tr, y_train_reviews)


In [ ]:
if not best_model_nn_reviews_is_saved:
    pickle.dump(best_model_nn_reviews, open('models/best_model_nn_reviews.sav', 'wb'))
    best_model_nn_reviews_is_saved = True
else:
    best_model_nn_reviews=pickle.load(open('models/best_model_nn_reviews.sav', 'rb'))

In [ ]:
y_preds_nn_reviews = best_model_nn_reviews.predict(X_valid_reviews_tr)

balanced_acc_shallow_nn_reviews = balanced_accuracy_score(y_valid_reviews, y_preds_nn_reviews)

In [ ]:
balanced_acc_shallow_nn_reviews

Reviews:
Baseline of shallow NNs (balanced accuracy = average of recalls for each class): __0.758__

#### News dataset

In [ ]:
available_activations = ['relu', 'logistic', 'tanh']
available_solvers = ['adam', 'lbfgs']
def objective(trial: optuna.trial.Trial):
    size_of_hidden_layer = trial.suggest_int('size_of_hidden_layer', 10, 100)
    alpha = trial.suggest_float('alpha', 0.0001, 0.001)
    activation = trial.suggest_categorical('activation', 
                                           choices=available_activations)
    solver = trial.suggest_categorical('solver', 
                                           choices=available_solvers)
    
    print(f"Trying: {size_of_hidden_layer}, {alpha}, {activation}, {solver}")
    model = MLPClassifier(hidden_layer_sizes=tuple([size_of_hidden_layer]),
                          alpha=alpha,
                          activation = activation,
                          max_iter=20,
                          solver=solver,
                          verbose=True,
                          learning_rate='adaptive',
                          random_state=42)
    model.fit(X_train_news_16th, y_train_news_16th)
    y_pred_probas = model.predict_proba(X_valid_news_tr)
    return log_loss(y_valid_news, y_pred_probas)

if not best_model_nn_news_is_saved:
    study_nn_news = optuna.create_study(direction='minimize')
    # optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study_nn_news.optimize(objective, n_trials=10,show_progress_bar=True,n_jobs=2)

    study_nn_news.best_params

best value: 0.3686714944409208 (log loss)
 
best params:
{'size_of_hidden_layer': 34,
 'alpha': 0.0004980605929484772,
 'activation': 'tanh',
 'solver': 'adam'}
 
last:
{'size_of_hidden_layer': 28, 'alpha': 0.0009033187102132166, 'activation': 'tanh', 'solver': 'adam'}

In [ ]:
if not best_model_nn_news_is_saved:
    best_model_nn_news = MLPClassifier(hidden_layer_sizes=(34,), 
                              alpha=0.0004980605929484772,
                              solver='adam',
                              activation='tanh',
                                random_state=42,
                               learning_rate='adaptive',
                               verbose=True,
                              max_iter=20,
                          )
    best_model_nn_news.fit(X_train_news_8th, y_train_news_8th)


In [45]:
if not best_model_nn_news_is_saved:
    pickle.dump(best_model_nn_news, open('models/best_model_nn_news.sav', 'wb'))
    best_model_nn_news_is_saved = True
else:
    best_model_nn_news=pickle.load(open('models/best_model_nn_news.sav', 'rb'))

In [43]:
y_preds_nn_news = best_model_nn_news.predict(X_valid_news_tr)

balanced_acc_shallow_nn_news = balanced_accuracy_score(y_valid_news, y_preds_nn_news)

In [44]:
balanced_acc_shallow_nn_news

0.8971229026985744

News Baseline of shallow NNs (balanced accuracy = average of recalls for each class): __0.92__

### SVM

#### Reviews dataset

In [ ]:
available_kernels = ['poly', 'sigmoid', 'linear']
def objective(trial: optuna.trial.Trial):
    kernel = trial.suggest_categorical('kernel', 
                                           choices=available_kernels)
    degree = trial.suggest_int('degree', 3,6)
    print(f"Trying: {kernel}")
    model = SVC(kernel=kernel,
                degree=degree,
                gamma='scale',
               verbose=True,
               probability=True,
               random_state=42)
    model.fit(X_train_reviews_tr, y_train_reviews)
    y_pred_probas = model.predict_proba(X_valid_reviews_tr)
    return log_loss(y_valid_reviews, y_pred_probas)

if not best_model_svm_reviews_is_saved:
    study_svm_reviews = optuna.create_study(direction='minimize')
    # optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study_svm_reviews.optimize(objective, n_trials=5,show_progress_bar=True,n_jobs=-1)

    study_svm_reviews.best_params

In [81]:
if not best_model_svm_reviews_is_saved:
    best_model_svm_reviews = SVC(kernel='linear', verbose=True)
    best_model_svm_reviews.fit(X_train_reviews_tr, y_train_reviews)

[LibSVM]

NameError: name 'y_preds_svm' is not defined

In [ ]:
if not best_model_svm_reviews_is_saved:
    pickle.dump(best_model_svm_reviews, open('models/best_model_svm_reviews.sav', 'wb'))
    best_model_svm_reviews_is_saved = True
else:
    best_model_svm_reviews=pickle.load(open('models/best_model_svm_reviews.sav', 'rb'))

In [82]:
y_preds_svm_reviews = best_model_svm_reviews.predict(X_valid_reviews_tr)

balanced_acc_svm_reviews = balanced_accuracy_score(y_valid_reviews, y_preds_svm_reviews)

In [83]:
balanced_acc_svm_reviews

0.7806725304721253

Balanced accuracy for reviews dataset with SVM: __0.78__

#### News dataset

In [ ]:
model = SVC(probability=True, random_state=42)
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel':['poly', 'sigmoid', 'linear', 'rbf']} 
  
randomized = RandomizedSearchCV(model, param_grid, refit = True, verbose = 3, random_state=42)

In [ ]:
randomized.fit(X_train_news_16th, y_train_news_16th)

In [ ]:
randomized.best_params_

In [ ]:
randomized.best_score_

In [ ]:

if not best_model_svm_news_is_saved:
    start=time.time()
    best_model_svm_news = SVC(kernel='linear', gamma=0.001, C=10, verbose=True, random_state=42)
    best_model_svm_news.fit(X_train_news_tr, y_train_news)
    end=time.time()

print("It took ", end-start, "seconds")

In [ ]:
if not best_model_svm_news_is_saved:
    pickle.dump(best_model_svm_news, open('models/best_model_svm_news.sav', 'wb'))
    print('dumped')
    best_model_nn_news_is_saved = True
# else:
#    best_model_svm_news=pickle.load(open('models/best_model_svm_news.sav', 'rb'))

In [82]:
y_preds_svm_news = best_model_svm_news.predict(X_valid_news_tr)

balanced_acc_svm_news = balanced_accuracy_score(y_valid_news, y_preds_svm_news)

In [83]:
balanced_acc_svm_news

0.7806725304721253

Balanced accuracy for news dataset with SVM: __0.912__

## Evaluating shallow models

## Tranining RNN architectures

### Transfer learning with BERT as a base model

### Architecture 2

## Evaluating RNN performance

## Comparison: RNNs vs Shallow